In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
import numpy as np
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [2]:
def initialize_service(api_token):
    QiskitRuntimeService.save_account(channel="ibm_quantum", token="deed0480b9f0aa55dcba0e6b87e880978deecf31964257d1b986d4968f4e4b980ef0b38c131d93bdde5f8b1648b48a986db7e6f02f3d15dd7f1289b7f147e2ba", overwrite=True)
    service = QiskitRuntimeService()
    return service

In [16]:
# Quantum Random Bit Generator using IBM Hardware
def generate_random_bits_hardware(api_token, qubits=100, shots=1000):
    # Initialize service
    service = initialize_service(api_token)
    
    # Select real quantum backend (for example, 'ibmq_montreal' or 'ibmq_brisbane')
    backend = service.backend("ibm_brisbane")

    
    # Create quantum and classical registers
    q = QuantumRegister(qubits, 'q')
    c = ClassicalRegister(qubits, 'c')
    circ = QuantumCircuit(qubits)
    
    # Apply Hadamard gate to all qubits to create superposition
    for i in range(qubits):
        circ.h(q[i])
    
    # Measure all qubits
    circ.measure_all()

    # Transpile the circuit for the backend
    compiled_circuit = transpile(circ, backend)
    
    # Initialize sampler and run the job on the selected backend
    sampler = Sampler(backend)

    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    isa = pm.run(circ)

    pub = (isa)
    job = sampler.run([pub], shots=shots)
    
    # Get the result
    result = job.result()
    
    # Extract counts and convert to binary digits
    counts = result[0].data.meas.get_counts()
    
    
    # Generate random bits from the counts
    random_bits = []
    for key in counts:
        random_bits.extend([key] * int(counts[key] * shots))
    
    return ''.join(random_bits)

In [17]:
# Main function to generate 100,000 binary digits
def main():
    api_token = "deed0480b9f0aa55dcba0e6b87e880978deecf31964257d1b986d4968f4e4b980ef0b38c131d93bdde5f8b1648b48a986db7e6f02f3d15dd7f1289b7f147e2ba"  # Replace with your IBM Quantum API token
    required_bits = 100000
    total_bits = []
    
    # Generate random bits in batches until we reach 100,000 bits
    while len(total_bits) < required_bits:
        new_bits = generate_random_bits_hardware(api_token, qubits=100, shots=1000)
        total_bits.extend(new_bits)
    
    # Truncate to exactly 100,000 bits
    total_bits = total_bits[:required_bits]
    
    # Save the random bits to a file
    with open('quantum_random_bits_100000.txt', 'w') as f:
        f.write(''.join(total_bits))
    
    print(f"Generated 100,000 random bits and saved to 'quantum_random_bits_100000.txt'.")

# Execute the main function
if __name__ == "__main__":
    main()

Generated 100,000 random bits and saved to 'quantum_random_bits_100000.txt'.
